In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os
import pandas as pd
from PIL import Image

In [ ]:
dataset_dir_img = "gdrive/MyDrive/flickr30k_images/flickr30k_images"

In [ ]:
image_data = []
for root, dirs, files in os.walk(os.path.join(dataset_dir_img, "images")):
    for file in files:
        # Check if the file is an image file
        if file.endswith((".jpg", ".jpeg", ".png")):
            # Construct the full file path
            file_path = os.path.join(root, file)

            # Load the image using PIL
            image = Image.open(file_path)
            image_data.append(image)

In [ ]:
dataset_dir_csv = "gdrive/MyDrive/flickr30k_images/results.csv"

In [ ]:
with open(dataset_dir_csv, 'r') as file:
    # Read the first line to get the column names
    columns = file.readline().strip().split('|')

    # Read remaining lines, split by the '|' separator, and construct the DataFrame
    data = [line.strip().split('|') for line in file]

    # Create a DataFrame from the parsed data with the extracted column names
    df = pd.DataFrame(data, columns=columns)

# Now you have the DataFrame `df` containing data from the CSV file
# You can access columns like `image_name`, `comment_number`, and `comment`
print(df.head())  # Example: print the first few rows of the DataFrame

       image_name  comment_number  \
0  1000092795.jpg               0   
1  1000092795.jpg               1   
2  1000092795.jpg               2   
3  1000092795.jpg               3   
4  1000092795.jpg               4   

                                             comment  
0   Two young guys with shaggy hair look at their...  
1   Two young , White males are outside near many...  
2   Two men in green shirts are standing in a yard .  
3       A man in a blue shirt standing in a garden .  
4            Two friends enjoy time spent together .  


In [ ]:
df.columns

Index(['image_name', ' comment_number', ' comment'], dtype='object')

In [ ]:
df = pd.read_csv(dataset_dir_csv, delimiter='|')

# Display the first few rows of the DataFrame
print(df.head())

       image_name  comment_number  \
0  1000092795.jpg               0   
1  1000092795.jpg               1   
2  1000092795.jpg               2   
3  1000092795.jpg               3   
4  1000092795.jpg               4   

                                             comment  
0   Two young guys with shaggy hair look at their...  
1   Two young , White males are outside near many...  
2   Two men in green shirts are standing in a yard .  
3       A man in a blue shirt standing in a garden .  
4            Two friends enjoy time spent together .  


In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    if isinstance(text, str):  # Check if text is a string
        # Lowercasing
        text = text.lower()

        # Tokenization
        tokens = word_tokenize(text)

        # Removing Punctuation
        tokens = [token for token in tokens if token not in string.punctuation]

        # Removing Stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]

        # Stemming
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

        return tokens
    else:
        return []  # Return an empty list for non-string inputs


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['preprocessed_comment'] = df[' comment'].apply(preprocess_text)


In [ ]:
df['preprocessed_comment']

0         [two, young, guy, shaggi, hair, look, hand, ha...
1         [two, young, white, male, outsid, near, mani, ...
2                     [two, men, green, shirt, stand, yard]
3                         [man, blue, shirt, stand, garden]
4                 [two, friend, enjoy, time, spent, togeth]
                                ...                        
158910    [man, short, hawaiian, shirt, lean, rail, pilo...
158911    [young, man, hang, side, boat, like, fog, roll...
158912    [man, lean, side, blue, white, boat, sit, bodi...
158913    [man, ride, small, boat, harbor, fog, mountain...
158914    [man, moor, blue, white, boat, hill, mist, bac...
Name: preprocessed_comment, Length: 158915, dtype: object

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

# Extract keywords for each row
keywords_per_row = []
for row in df['preprocessed_comment']:
    # Convert preprocessed comment to string
    comment_str = ' '.join(row)

    # Fit the vectorizer and transform the comment
    tfidf_matrix = tfidf_vectorizer.fit_transform([comment_str])

    # Get feature names (words)
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Get top N keywords based on highest TF-IDF scores
    top_keywords_indices = tfidf_matrix.sum(axis=0).argsort()[0, ::-1][:5]
    top_keywords = [feature_names[idx] for idx in top_keywords_indices]

    # Store keywords for this row
    keywords_per_row.append(top_keywords)

# Add extracted keywords as a new column in the DataFrame
df['keywords'] = keywords_per_row

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
df['keywords']

In [ ]:
top_keywords_tfidf